In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
import re
import time
from scipy import stats

import warnings
warnings.filterwarnings("ignore")
import scipy.stats

#Quality of image:
%config InlineBackend.figure_format = 'svg'
import os

pd.set_option('display.max_columns', 100)

# Import my own functions from utils file:
import import_ipynb
from utils_Rental_Cost import *

importing Jupyter notebook from utils_Rental_Cost.ipynb


In [2]:
dataDirectory = os.path.join(os.path.dirname(os.getcwd()),'Rental_Cost\\data')
imageDirectory = os.path.join(os.path.dirname(os.getcwd()),'Rental_Cost\\images\\Preparation')

In [3]:
df = pd.read_csv(dataDirectory + "\houses_to_rent.csv")
print(df.shape)
df.head(2)

(10692, 13)


,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa (R$),rent amount (R$),property tax (R$),fire insurance (R$),total (R$)
0,São Paulo,70,2,1,1,7,acept,furnished,2065,3300,211,42,5618
1,São Paulo,320,4,4,0,20,acept,not furnished,1200,4960,1750,63,7973


### Features of the dataset: 
**Unnamed: 0** Incremental id, for prediction puroses we do not need it.
<br>**city:** City where the home is located
<br>**area:** The area of the home,
<br>**rooms:** The count of rooms of the home,
<br>**bathroom:** The count of bathrooms of the home,
<br>**parking spaces:** The count of parking spaces belonged to the home,
<br>**floor:** The floor of the building,
<br>**animal:** Is it animal friendly?
<br>**furniture:** Is it furnished or not?
<br>**hoa** Homeowners association tax,
<br>**property tax:** Monthly property tax,
<br>**property tax:** Homeowners association tax,
<br>**rent amount:** Monthly rental price,
<br>**fire insurance:** Monthly fire insurance payment,
<br>**total** Total value of monthly payment

**1) Renaming columns**

In [4]:
df.rename(columns={'parking spaces': 'parking', 
                   'animal': 'petFriendly',
                   'furniture': 'furnished',
                   'rent amount (R$)': 'rent',
                   'property tax (R$)': 'tax',
                   'fire insurance (R$)': 'fireInsurance',
                   'hoa (R$)': 'hoa',
                   'total (R$)': 'total'
                  },inplace= True)

df.head(2)

,city,area,rooms,bathroom,parking,floor,petFriendly,furnished,hoa,rent,tax,fireInsurance,total
0,São Paulo,70,2,1,1,7,acept,furnished,2065,3300,211,42,5618
1,São Paulo,320,4,4,0,20,acept,not furnished,1200,4960,1750,63,7973


**2) Checking duplicates**

### Data Preparation:
- Checking duplicates,
- Checking missing values,
- Renaming columns,
- Deleting "Unnamed:0" column,
- Modifying city column
- Deleting **"R$ and comma"** in 5 financial columns: (hoa, rent amount, property tax, fire insurance, total),
- Converting string columns into numerical columns,
- Check if there is some non-numerical input within numerical columns,
- Looking for miss values,
- Looking for outliers,
- Checking multi colinearity

In [5]:
duplicated = df.duplicated()
duplicated = duplicated[duplicated.values==True]
duplicated.shape

(358,)

### Insight: 
- There are 358 duplicated rows. 
- However, according to the fact that the data set includes rental homes, it is possible that there are similar records. 
- For example, similar unit in a tall bauilding.
- So, we can keep them.

**3) Checking missing values**

In [6]:
df.isnull().sum()

city             0
area             0
rooms            0
bathroom         0
parking          0
floor            0
petFriendly      0
furnished        0
hoa              0
rent             0
tax              0
fireInsurance    0
total            0
dtype: int64

In [7]:
metadata = make_metadata(df)

catFeatures, catCnt, cat_df = make_typedf(df, metadata, 'categorical')
ordFeatures, ordCnt, ord_df = make_typedf(df, metadata, 'ordinal')
numFeatures, numCnt, num_df = make_typedf(df, metadata, 'numeric')
binFeatures, binCnt, bin_df = make_typedf(df, metadata, 'binary')

metadata

,role,type,dtype,uniqueCnt
varname,,,,
city,input,categorical,object,5
area,input,ordinal,int64,517
rooms,input,ordinal,int64,11
bathroom,input,ordinal,int64,10
parking,input,ordinal,int64,11
floor,input,categorical,object,35
petFriendly,input,categorical,object,2
furnished,input,categorical,object,2
hoa,input,ordinal,int64,1679


**5) Preparation of numerical columns:**
- Converting string data into numeric,
- Check if there is some non-numerical data within numerical columns,

In [8]:
check_numericalStatus(df, ordFeatures)

Dataframe is verified!
All columns are pure numerical.


[]

In [9]:
catFeatures

['city', 'floor', 'petFriendly', 'furnished']

**5) Modifying of Floor Feature:**
- It is considered categorical.
- We have to modify it.

In [10]:
non_num_index = find_nonNumerical_percent(df, 'floor');

rexxx
23.02% of the "floor" column is not numerical.
The distribution is as follow:
-    2461
Name: floor, dtype: int64
********************************************************


In [11]:
df.loc[non_num_index].head(3)

,city,area,rooms,bathroom,parking,floor,petFriendly,furnished,hoa,rent,tax,fireInsurance,total
5,São Paulo,376,3,3,7,-,acept,not furnished,0,8000,834,121,8955
15,Campinas,330,4,6,6,-,acept,furnished,680,8000,328,121,9129
19,São Paulo,600,4,5,6,-,acept,not furnished,0,12000,9500,181,21680


In [12]:
restIndex = ~df.index.isin(non_num_index)
dfTest = df.loc[restIndex]
cityLst = dfTest['city'].unique().tolist()
cityLst

['São Paulo', 'Porto Alegre', 'Rio de Janeiro', 'Campinas', 'Belo Horizonte']

**Imputing floor feature based on the most frequent data:**

In [13]:
cityMode = dfTest.groupby('city')['floor'].agg(pd.Series.mode)
cityMode

city
Belo Horizonte    1
Campinas          1
Porto Alegre      2
Rio de Janeiro    2
São Paulo         1
Name: floor, dtype: object

### Before imputing:

In [14]:
df.loc[df['city']=='São Paulo', 'floor'].value_counts().head()

-    1614
1     542
2     350
3     332
5     307
Name: floor, dtype: int64

### After imputing:

In [15]:
for cityName in cityLst:
    df.loc[(df['city']==cityName) & (df['floor']=='-'), 'floor'] = str(cityMode[cityName])
    
df.loc[df['city']=='São Paulo', 'floor'].value_counts().head()

1    2156
2     350
3     332
5     307
6     299
Name: floor, dtype: int64

In [16]:
df['floor'] = df['floor'].astype(int)

**Updating MetaData:**

In [17]:
metadata = make_metadata(df)

catFeatures, catCnt, cat_df = make_typedf(df, metadata, 'categorical')
ordFeatures, ordCnt, ord_df = make_typedf(df, metadata, 'ordinal')
numFeatures, numCnt, num_df = make_typedf(df, metadata, 'numeric')
binFeatures, binCnt, bin_df = make_typedf(df, metadata, 'binary')

# Let us take a look at cat features:
catFeatures

['city', 'petFriendly', 'furnished']

**Exporting dataset for future using:**

In [18]:
df.to_csv(dataDirectory + "\\rental_houses_Tohid.csv", index=False)